# YOLOv9 ONNX conversion
![YOLOv9](https://s4.itho.me/sites/default/files/styles/picture_size_large/public/field/image/perf.png?itok=wxwXwm7c)

In [8]:
%cd /content
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9
!pip3 install -r requirements.txt

/content
fatal: destination path 'yolov9' already exists and is not an empty directory.
/content/yolov9


## Convert model


In [9]:
%cd /content/yolov9
!pip3 install onnx>=1.10.0

model_name = 'yolov9-e' #@param ["yolov9-c", "yolov9-e"]
input_width = 640 #@param {type:"slider", min:32, max:4096, step:32}
input_height = 480 #@param {type:"slider", min:32, max:4096, step:32}

import os.path
torch_model_name = f'{model_name}-converted.pt' # converted version is remove auxiliary branch
if not os.path.exists(torch_model_name):
  !wget https://github.com/WongKinYiu/yolov9/releases/download/v0.1/{torch_model_name}

import torch
from pathlib import Path
from export import export_onnx, attempt_load
model = attempt_load(torch_model_name, device='cpu', inplace=True, fuse=True)
model.float()
model.eval()
for name, module in model.named_modules():
  if hasattr(module, 'fuse_convs'):
    module.fuse_convs()
    module.forward = module.forward_fuse
  module.export = True

im = torch.randn(1, 3, input_height, input_width)

/content/yolov9


Fusing layers... 
gelan-e summary: 690 layers, 57345920 parameters, 50926448 gradients, 189.0 GFLOPs


## **Download method 1**

https://docs.ultralytics.com/vi/integrations/onnx/#onnx-and-onnx-runtime

In [14]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO(f"{torch_model_name}")

# Export the model to ONNX format
model.export(format="onnx")  # creates 'yolov8n.onnx'

# Load the exported ONNX model
onnx_model = YOLO(f'{model_name}-converted.onnx')

# Run inference
results = onnx_model("https://ultralytics.com/images/bus.jpg")

Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)


Fusing layers... 
gelan-e summary: 690 layers, 57345920 parameters, 0 gradients, 189.0 GFLOPs



PyTorch: starting from 'yolov9-e-converted.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 84, 8400), ()) (111.7 MB)

ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success ✅ 17.0s, saved as 'yolov9-e-converted.onnx' (219.2 MB)

Export complete (29.9s)
Results saved to /content/yolov9
Predict:         yolo predict task=detect model=yolov9-e-converted.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov9-e-converted.onnx imgsz=640 data=None  
Visualize:       https://netron.app
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov9-e-converted.onnx for ONNX Runtime inference...

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /content/yolov9/bus.jpg: 640x640 4 persons, 1 bus, 6668.4ms
Speed: 8.6ms preprocess, 6668.4ms inference, 2.2ms post

In [15]:
from google.colab import files

files.download(f'{model_name}-converted.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Download model 2**

In [16]:
# export_onnx(model, im, Path(model_name+".onnx"), 12, dynamic=False, simplify=True)
# %cd /content/yolov9
# from google.colab import files

# files.download(f'{model_name}.onnx')